In [1]:
import os
import sys
import subprocess
import datetime as dt
import itertools
import numpy as np
import xarray as xr
from dask.distributed import Client, LocalCluster
import dask.array as da
import matplotlib.pyplot as plt
import cmocean
from MHW_daily_calcs import *

In [2]:
fin=xr.open_dataset(fnameCanESMdaily(mdirC5,1993,1,1,0),decode_times=False)

In [6]:
def _add_dims(arr,tarr): # add trailing dimensions of length 1 until dimension length of tarr is reached
    while len(np.shape(arr))<len(np.shape(tarr)):
        arr=np.expand_dims(arr,-1)
    return arr

In [7]:
def gsmooth(v,x,L,axis=0):
    if not axis==0:
        v=np.moveaxis(v,axis,0)
    if len(np.shape(v))==1:
        v=np.expand_dims(v,axis=1)
    fil=np.empty(np.shape(v))
    s=L/2.355
    for ind, t in enumerate(x):
        diff=np.abs(x-t)
        wgt=_add_dims(np.array([np.exp(-.5*x**2/s**2) if x<=3*L else 0.0 for x in diff]),v)
        fil[ind,...]=np.divide(np.nansum(wgt*v,0),np.nansum(wgt*~np.isnan(v),0),
                               out=np.nan*da.array(np.ones(np.shape(v)[1:])),
                               where=np.nansum(wgt*~np.isnan(v),0)>0)
    if not axis==0:
        fil=np.moveaxis(fil,0,axis)
    return fil

In [8]:
%time
test=gsmooth(fin.tso.values,fin.leadtime.values,10*24,1)

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.87 µs


In [ ]:
np.shape(test),np.shape(fin.tso.values)